In [ ]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import split, col
from pyspark.sql.types import TimestampType

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

geoEventProcessing = spark.readStream\
                          .format("socket")\
                          .option("host", "localhost")\
                          .option("port", 9999)\
                          .load()\
                          .select(split("value",'\|').alias("fields"))\
                          .withColumn("timestamp_str",col("fields").getItem(0))\
                          .withColumn("timestamp",col("timestamp_str").cast(TimestampType()))\
                          .withColumn("driverName",col("fields").getItem(4))\
                          .withColumn("violation", col("fields").getItem(7))\
                          .where("violation!='Normal'")\
                          .groupBy("driverName", "violation")\
                          .count()\
                          .orderBy(col("count").desc())\
                          .limit(5)\
                          .writeStream\
                          .format("console")\
                          .outputMode("complete")\
                          .trigger(processingTime='10 seconds')\
                          .start()\
                          .awaitTermination()